In [54]:
%load_ext autoreload
%autoreload 2

In [ ]:
!pip install llama-index llama-hub unstructured lxml cohere -qU

In [67]:
!pip install environ

  Using cached environ-1.0-py3-none-any.whl

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
!pip install unstructured==0.10.18

In [2]:
import nest_asyncio

nest_asyncio.apply()

In [69]:
import os
from dotenv import load_dotenv

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['SERPER_API_KEY'] = os.getenv('SERPER_API_KEY')

In [ ]:
from llama_index.llama_pack import download_llama_pack

EmbeddedTablesUnstructuredRetrieverPack = download_llama_pack('EmbeddedTablesUnstructuredRetrieverPack', './embedded_tables_unstructured_pack')

In [ ]:
import importlib
importlib.reload(EmbeddedTablesUnstructuredRetrieverPack)


In [ ]:
from embedded_tables_unstructured_pack.base import EmbeddedTablesUnstructuredRetrieverPack

EmbeddedTablesUnstructuredRetrieverPack??

In [ ]:
dir(EmbeddedTablesUnstructuredRetrieverPack)

In [ ]:
!wget https://github.com/pdf2htmlEX/pdf2htmlEX/releases/download/v0.18.8.rc1/pdf2htmlEX-0.18.8.rc1-master-20200630-Ubuntu-bionic-x86_64.deb

In [ ]:
!sudo apt install "./pdf2htmlEX-0.18.8.rc1-master-20200630-Ubuntu-bionic-x86_64.deb" -y

In [ ]:
!wget https://www.combustion911.com/support/documentation/actuators/ic-20-ic-50-actuators_brochure.pdf -o ic-20-ic-50-datasheet.pdf

In [ ]:
import subprocess

def convert_pdf_to_html(pdf_path, html_path):
    command = f'pdf2htmlEX {pdf_path} --dest-dir {html_path}'
    subprocess.call(command, shell=True)

input_pdf = 'ic-20-ic-50-actuators_brochure.pdf'
output_pdf = 'ic-20-ic-50-actuators_brochure'

convert_pdf_to_html(input_pdf, output_pdf)

In [ ]:
embedded_tables_unstructured_pack = EmbeddedTablesUnstructuredRetrieverPack(
    f'{output_pdf}/{output_pdf}.html',
    nodes_save_path = f'{output_pdf}.pkl'
)

In [ ]:
modules = embedded_tables_unstructured_pack.get_modules()
display(modules)

In [ ]:
prompt = """
You are manufacturer sales manager. You help clients answer on question about technical specification of factory parts.
You have deep knowledge about physics, engineering and constructing manufacturing parts.

As a manufacturer sales manager please what is valtage of Actuator IC-20?
"""

response = embedded_tables_unstructured_pack.run('What is the difference between ic-20 and ic-50, build compare table?')
print(response.response)

In [ ]:
print(response.response)

In [ ]:
response = embedded_tables_unstructured_pack.run("""
This document is specification of actuators from manufacturer or seller.
Please say me manufacturer details like address, details and other of actuators.""")
response.response

In [ ]:
response.response

In [ ]:
from llama_index import VectorStoreIndex
from llama_index.node_parser import UnstructuredElementNodeParser
from typing import Dict, Any
from llama_index.llama_pack.base import BaseLlamaPack
from llama_index.retrievers import RecursiveRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.readers.file.flat_reader import FlatReader
from pathlib import Path
from typing import Optional
import os
import pickle

In [ ]:
# def __init__(
#         self,
#         html_path: str,
#         nodes_save_path: Optional[str] = None,
#         **kwargs: Any,
#     ) -> None:
#         """Init params."""
#         self.reader = FlatReader()

#         docs = self.reader.load_data(Path(html_path))

#         self.node_parser = UnstructuredElementNodeParser()
#         if nodes_save_path is None or not os.path.exists(nodes_save_path):
#             raw_nodes = self.node_parser.get_nodes_from_documents(docs)
#             pickle.dump(raw_nodes, open(nodes_save_path, "wb"))
#         else:
#             raw_nodes = pickle.load(open(nodes_save_path, "rb"))

#         base_nodes, node_mappings = self.node_parser.get_base_nodes_and_mappings(
#             raw_nodes
#         )
#         # construct top-level vector index + query engine
#         vector_index = VectorStoreIndex(base_nodes)
#         vector_retriever = vector_index.as_retriever(similarity_top_k=1)
#         self.recursive_retriever = RecursiveRetriever(
#             "vector",
#             retriever_dict={"vector": vector_retriever},
#             node_dict=node_mappings,
#             verbose=True,
#         )
#         service_context = ServiceContext.from_defaults(
#           llm=OpenAI(model="gpt-4")
#         )

#         self.query_engine = RetrieverQueryEngine.from_args(
#           self.recursive_retriever,
#           service_context=service_context
#         )

In [ ]:
output_name = 'ic-20-ic-50-actuators_brochure'

In [ ]:
html_path = f'./{output_name}/{output_name}.html'
nodes_save_path = f'{output_name}.pkl'

In [ ]:
html_path

In [ ]:
reader = FlatReader()

In [ ]:
docs = reader.load_data(Path(html_path))

In [ ]:
UnstructuredElementNodeParser?

In [ ]:
node_parser = UnstructuredElementNodeParser()
node_parser

In [ ]:
raw_nodes = pickle.load(open(nodes_save_path, "rb"))

In [ ]:
import pprint

for node in raw_nodes:
    node.text = node.text.replace('\ue601', ' ')

In [ ]:
pprint.pprint(raw_nodes[0].__dict__)

In [ ]:
base_nodes, node_mappings = node_parser.get_base_nodes_and_mappings(
            raw_nodes
        )

In [ ]:
# base_nodes

In [ ]:
vector_index = VectorStoreIndex(base_nodes)
# vector_retriever = vector_index.as_retriever(similarity_top_k=1)

vector_retriever = vector_index.as_retriever(similarity_top_k=3)
vector_query_engine = vector_index.as_query_engine(similarity_top_k=3)

recursive_retriever = RecursiveRetriever(
            "vector",
            retriever_dict={"vector": vector_retriever},
            node_dict=node_mappings,
            verbose=True,
        )

In [ ]:
service_context = ServiceContext.from_defaults(
          llm=OpenAI(model="gpt-4")
        )

query_engine = RetrieverQueryEngine.from_args(
          recursive_retriever,
          service_context=service_context
        )

In [ ]:
from llama_index import ServiceContext
from llama_index.llms import OpenAI

In [ ]:
response = query_engine.query(prompt)


In [ ]:
response.response

In [ ]:
import pprint


pprint.pprint(base_nodes[0].__dict__)

In [ ]:
UnstructuredElementNodeParser??

## Querying Google

In [3]:
import json
import requests

In [4]:
def search(query, country: str = 'us', page: int = 1):
    url = "https://google.serper.dev/search"
    
    payload = json.dumps({
      "q": query,
      "gl": country,
      "page": page
    })

    headers = {
        'X-API-KEY': os.environ['SERPER_API_KEY'],
        'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    return json.loads(response.text)

In [ ]:
query1 = 'CONTRINEX  DW-AS-503-P12-627  Nr. 532157'
query2 = 'Actuator IC-20'

results = []

for i in range(2):
    results += search(query2, page = i + 1)['organic']

results

In [ ]:
# query_response_json = json.loads(search_response)

print(json.dumps(results, indent=2))

In [9]:
['{price} // {title}'.format(price=resp['currency'] + str(resp['price']), title=resp['title']) for resp in results if 'price' in resp]

links_with_price = [resp for resp in results if 'price' in resp]
links_with_price

[{'title': 'Kromschroder Actuator IC 20-30W3E | 88300076 - Combustion 911',
  'link': 'https://www.combustion911.com/products/actuators/ic-20/ic-20-30w3e.html',
  'snippet': 'Kromschroder Actuator IC 20 is for applications that require precise, controlled rotary movement between 0 and 90 degrees. Control ratios up to 1:10.',
  'currency': '$',
  'price': 868,
  'position': 3},
 {'title': 'Honeywell Kromschroder IC20 and IC40 Valve Actuators - Lesman: Instruments',
  'link': 'https://www.lesman.com/honeywell-kromschroder-ic20-and-ic40-valve-actuators-for-bv-butterfly-valves',
  'snippet': "Honeywell Kromschroder's IC20 and IC40 valve actuators are used for control applications that require rotary movements between 0° and 90°.",
  'currency': '$',
  'price': 452,
  'position': 6},
 {'title': 'Kromschroder IC Actuators - KromAmericas',
  'link': 'https://www.kromamericas.com/Catalog/actuators/ic',
  'snippet': 'The Kromschroder actuators are designed for all applications that require prec

In [ ]:
query_response_json['organic']

## Parse Web Page

In [29]:
from bs4 import BeautifulSoup
import requests
import pickle
from llama_index.node_parser import UnstructuredElementNodeParser
from llama_index.readers.file.flat_reader import FlatReader
from pathlib import Path
import pprint

In [19]:
response = requests.get(links_with_price[0]['link'])

soup = BeautifulSoup(response.content, 'html.parser')

In [13]:
with open('./assets/price.html', 'wb') as f:
    f.write()

In [ ]:
reader = FlatReader()
docs = reader.load_data(Path('./assets/price.html'))

node_parser = UnstructuredElementNodeParser()
raw_nodes = node_parser.get_nodes_from_documents(docs)

In [ ]:
pprint.pprint(raw_nodes[0].__dict__)

In [31]:
base_nodes, node_mappings = node_parser.get_base_nodes_and_mappings(raw_nodes)

In [ ]:
base_nodes

In [ ]:
node_mappings


### Pickle

In [15]:
with open("output.pkl", "wb") as pkl_file:
    pickle.dump(soup, pkl_file)

In [ ]:
pickled_soup = pickle.dumps(soup)


pickle.loads(pickled_soup)

In [14]:
raw_nodes = pickle.load(open('./assets/price.html', 'rb'))

raw_nodes

UnpicklingError: invalid load key, '<'.

In [56]:
from bs4 import BeautifulSoup
import re

# soup = BeautifulSoup(response.text, 'html.parser')

sentence_pattern = r'(?<=\.|\?|\!)\s+'
pattern = r'\n+'
# Remove new lines and tabulations before and after each sentence
# cleaned_text = re.sub(sentence_pattern, ' ', text)

# Remove <script> and <style> elements
for script in soup(['script', 'style', 'a']):
    script.extract()

# Select only blocks that have text content
# text_blocks = [re.sub(pattern, '\n', block.get_text()) for block in soup.find_all(['p', 'div', 'span']) if block.get_text(strip=True)]

# # Print the text content of selected blocks
# for block_text in text_blocks:
#     if len(block_text) > 0:
#         print(block_text)

# # print(f'The block text: """{text_blocks[0]}"""')

# text_blocks

# self.reader = FlatReader()

#         docs = self.reader.load_data(Path(html_path))

#         self.node_parser = UnstructuredElementNodeParser()
#         if nodes_save_path is None or not os.path.exists(nodes_save_path):
#             raw_nodes = self.node_parser.get_nodes_from_documents(docs)
#             pickle.dump(raw_nodes, open(nodes_save_path, "wb"))
#         else:
#             raw_nodes = pickle.load(open(nodes_save_path, "rb"))


In [ ]:
with open("output.pkl", "wb") as pkl_file:
    pickle.dump(soup, pkl_file)

In [8]:
# reader = FlatReader()
node_parser = UnstructuredElementNodeParser()
raw_nodes = pickle.loads(response.content)

base_nodes, node_mappings = node_parser.get_base_nodes_and_mappings(
            raw_nodes
        )

NameError: name 'response' is not defined

In [ ]:
reader = FlatReader()
# docs = reader.load_data(response.content)

reader??

In [ ]:
response.content

## Vectorize Html Page

In [44]:
from llama_index import VectorStoreIndex
from llama_index.retrievers import RecursiveRetriever
from llama_index import ServiceContext
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.llms import OpenAI

In [46]:
similarity_top_k = 3

In [55]:
vector_index = VectorStoreIndex(base_nodes)

vector_retriever = vector_index.as_retriever(similarity_top_k=similarity_top_k)
# vector_query_engine = vector_index.as_query_engine(similarity_top_k=3)

recursive_retriever = RecursiveRetriever(
            "vector",
            retriever_dict={"vector": vector_retriever},
            node_dict=node_mappings,
            verbose=True,
)
service_context = ServiceContext.from_defaults(llm=OpenAI(model="gpt-3.5-turbo"))

query_engine = RetrieverQueryEngine.from_args(recursive_retriever, service_context=service_context)

In [59]:
response = query_engine.query(f'Give technical specification of {query2}')

Retrieving with query id None: Give technical specification of Actuator IC-20
Retrieving text node: Applications (click on image for close-up view)

IC 20..E, continuous control

For processes that require high temperature accuracy and low circulation in the furnace. The actuator IC 20..E is controlled by a 4 – 20 mA, 0 – 20 mA or 0 – 10 V signal.

Specifications
Retrieved node with id, entering: id_41_table
Retrieving with query id id_41_table: Give technical specification of Actuator IC-20
Retrieved node with id, entering: id_46_table
Retrieving with query id id_46_table: Give technical specification of Actuator IC-20


In [61]:
print(response.response)

The technical specifications of Actuator IC-20 are as follows:
- Mains voltage: 230 V AC, -15/+10% 50/60 Hz
- Power consumption: 4.9 VA at 50 Hz, 5.8 VA at 60 Hz
- Angle of rotation: 0 – 90°, adjustable
- Holding torque: torque
- Switching power of the position switches: 
  - Voltage: 125 V~ 2 A, 250 V~ 2 A, <30 V= 2 A, <50 V= 1 A, <75 V= 0.75 A, <125 V= 0.5 A, <250 V= 0.25 A, 12…30 V~/= 100 mA
  - Resistive load: 2 A, 0.5 A, 2 A
  - Incandescent lamp load: 0.5 A, 0.4 A, 0.3 A, 0.2 A, 0.1 A
  - Inductive load: 2 A, 2 A, 2 A, 1 A, 0.75 A, 0.5 A, 0.25 A
- Power consumption: Terminals 1, 2 and 5: 4.9 VA at 50 Hz, 5.8 VA at 60 Hz, Terminal 3: 8.4 VA at 50 Hz, 9.5 VA at 60 Hz, in total not exceeding: 8.4 VA at 50 Hz, 9.5 VA at 60 Hz
- Feedback output: Galvanically isolated, load impedance max. 500 Ω
- Input: Galvanically isolated, 4 (0) – 20 mA: load impedance switchable from 50 Ω to 250 Ω, 0 – 10 V: 100 kΩ input resistance
- Enclosure: IP 65 pursuant to IEC 529
- Protection class: Impact-r

### JSON Reader

In [70]:
from llama_index.readers import JSONReader
from openai import OpenAI
import os

In [71]:
query2 = 'Actuator IC-20 datasheet'

results = []

for i in range(2):
    results += search(query2, page = i + 1)['organic']

links_with_price = [resp for resp in results if 'price' in resp]

In [75]:
joined_links = '\n\n'.join(['Title: {title}\nSnippet: {snippet}\nPrice: {currency}{price}'.format(
    title=price_link['title'],
    snippet=price_link['snippet'],
    currency=price_link['currency'],
    price=price_link['price']
)  for price_link in links_with_price])

print(joined_links)

Title: Kromschroder Actuator IC 20-30W3E | 88300076 - Combustion 911
Snippet: Actuator IC 20-30W3E · Running time: 30 seconds · Mains Voltage: 230 V AC, -15/+10% 50/60 Hz · Torque: 3.0Nm · Continuous Control: 0 (4) – 20 mA, 0 – 10 V · For ...
Price: $868

Title: KS88300080 - IC 20-30Q3E - KromAmericas
Snippet: The actuators are designed for all applications that require precise, controlled rotary movement between 0° and 90°. In conjunction with a control element, ...
Price: $1.324

Title: IC 20-60W3E | Krom Schroder | Actuators IC - NEX Instrument Inc
Snippet: IC 20-60W3E | Krom Schroder | Actuators IC, Best Price in Town, Guaranteed Low Price!
Price: $1.258

Title: Honeywell Kromschroder IC20 and IC40 Valve Actuators | Id - SOS Safety International Inc.
Snippet: Honeywell Kromschroder's IC20 and IC40 valve actuators can be mounted directly onto BV butterfly valves for applications that require rotary movements ...
Price: $452

Title: Kromschroder IC20-30W3E 230v 50hZ Actuator - Indust

In [64]:
openai = OpenAI(model='gpt-4')

In [66]:
openai?

Type:            OpenAI
String form:     callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7fe87b6d7c10> model='gpt <...> M95L2MAJQG7jm7u7T3BlbkFJnPYMsCoR2gITOVpikeCz' api_base='https://api.openai.com/v1' api_version=''
File:            ~/projs/components_agent_sales/venv/lib/python3.11/site-packages/llama_index/llms/openai.py
Docstring:       <no docstring>
Class docstring: LLM interface.
Init docstring: 
Create a new model by parsing and validating input data from keyword arguments.

Raises ValidationError if the input data cannot be parsed to form a valid model.

In [84]:

client = OpenAI(api_key = os.getenv('OPENAI_API_KEY'))

response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "You are manufacturer details sales manager. You understand technical specification and trying to find the best offer on market in proportion price/qualuty. Your work is accept the market infromation and build tables that shows comparance between manufacturer details."},
        {"role": "user", "content": f"Hey, I need to buy Actuator IC-20, here is different offers:\n{joined_links}. \n\nCould you please choose relevant variants and build comparance table. Put result into markdown format."}
    ],
    stream=False
)
response

ChatCompletion(id='chatcmpl-8UrL7WPCh21QOQxtQ0XE0i4D2RDGB', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Sure, here is the comparison table for Actuator IC-20 from different manufacturers and sellers:\n\n| Seller | Model | Description | Price |\n|---|---|---|---|\n| Combustion 911 | Kromschroder Actuator IC 20-30W3E | Running time: 30 seconds, Mains Voltage: 230 V AC, Torque: 3.0Nm, Continuous Control: 0 (4) – 20 mA, 0 – 10 V | $868 |\n| KromAmericas | KS88300080 - IC 20-30Q3E | Designed for precise, controlled rotary movement between 0° and 90°. Compatible with a control element | $1.324 |\n| NEX Instrument Inc | IC 20-60W3E | Designed for precise rotary movements, Claimed Best Price in Town | $1.258 |\n| SOS Safety International Inc. | Honeywell Kromschroder IC20 | Can be mounted directly onto BV butterfly valves for rotary movements applications | $452 |\n| Industrial Boilerhouse Supplies | Kromschroder IC20-30W3E 230v 50hZ | Upgrade combusti

In [80]:
from openai import OpenAI

client = OpenAI(api_key = os.getenv('OPENAI_API_KEY'))

stream = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "You are manufacturer details sales manager. You understand technical specification and trying to find the best offer on market in proportion price/qualuty. Your work is accept the market infromation and build tables that shows comparance between manufacturer details."},
        {"role": "user", "content": f"Hey, I need to buy Actuator IC-20, here is different offers:\n{joined_links}. \n\nCould you please choose relevant variants and build comparance table. Put result into markdown format."}
    ],
    stream=False,
)
stream

ChatCompletion(id='chatcmpl-8UrIiRMSJzCOTr5ugPdvACNdK7cV3', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content="Sure, I'll choose the relevant options for you and make a comparison table in markdown format. \n\n| Vendor | Product title | Key features | Price in $ |\n|---|---|---|---|\n| Combustion 911 | Kromschroder Actuator IC 20-30W3E  | Running time: 30 seconds, Mains Voltage: 230 V AC, Torque: 3.0Nm, Continuous Control: 0 (4) – 20 mA, 0 – 10 V | 868 |\n| KromAmericas | KS88300080 - IC 20-30Q3E  | Designed for applications that require precise, controlled rotary movement between 0° and 90° | 1324 |\n| NEX Instrument Inc | IC 20-60W3E  | Best Price in Town, Guaranteed Low Price! | 1258 |\n| SOS Safety International Inc. | Honeywell Kromschroder IC20 and IC40 Valve Actuators | Can be mounted directly onto BV butterfly valves for applications that require rotary movements | 452 |\n| Industrial Boilerhouse Supplies | Kromschroder IC20-30W3E 230v 50hZ Ac

In [83]:
print(stream.choices[0].message.content)

Sure, I'll choose the relevant options for you and make a comparison table in markdown format. 

| Vendor | Product title | Key features | Price in $ |
|---|---|---|---|
| Combustion 911 | Kromschroder Actuator IC 20-30W3E  | Running time: 30 seconds, Mains Voltage: 230 V AC, Torque: 3.0Nm, Continuous Control: 0 (4) – 20 mA, 0 – 10 V | 868 |
| KromAmericas | KS88300080 - IC 20-30Q3E  | Designed for applications that require precise, controlled rotary movement between 0° and 90° | 1324 |
| NEX Instrument Inc | IC 20-60W3E  | Best Price in Town, Guaranteed Low Price! | 1258 |
| SOS Safety International Inc. | Honeywell Kromschroder IC20 and IC40 Valve Actuators | Can be mounted directly onto BV butterfly valves for applications that require rotary movements | 452 |
| Industrial Boilerhouse Supplies | Kromschroder IC20-30W3E 230v 50hZ Actuator | Designed for precision and efficiency, this 30W3E 230V 50Hz actuator delivers reliable | 1573 (approx, converted from £1.171) |
